## Downsampling Audio to Desired Sample Rate & Saving FilePaths of New AudioFiles & associated TextGrids to Target Directory or Subdirectories of Target Directory

Assumptions:
1. Working in directory containing the Dataset (.wav files and associated TextGrids)
> Have an alternate workflow if the Dataset is already split into directories of Train, Test, & Val
2. Each .wav file and associated .TextGrid file have the exact same filename, but differ in their extension (.wav vs. .TextGrid)
Outputs:
1. New Directory in current directory containing the resampled audio & associated TextGrid for each file
2. Dataset containing names of TextGrids which can be concatenated with other dataframes containing information such as Gender, Classification Target Class, etc. to be saved as a .CSV and input into the SpectrifyDS_Creation notebook for creating a chunk mapping.

### Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive') # Allows connection to google drive for file access

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/Audio_SR_22050/
%pwd

/content/drive/MyDrive/Audio_SR_22050


'/content/drive/MyDrive/Audio_SR_22050'

In [3]:
!pip install soundfile

In [10]:
import pandas as pd
import scipy
import os
import numpy as np

### Saving Downsampled Audio Files & Associated TextGrids to Subdirectories of Target Directory if Dataset is split within multiple directories or directly to Target Directory if no split has occurred

In [16]:
def create_df_files(dir_name):

  # Creating list of all TextGrid Filepaths
  tg_files = []

  if isinstance(dir_name, str): # If working with a single directory with saved audio files & TextGrids
    for filename in os.listdir(dir_name):
      if filename.endswith(".TextGrid"):
        tg_files.append(dir_name + "/" + filename)
  elif isinstance(dir_name, list): # If working with multiple directories with saved audio files & TextGrids
    for ds_type in dir_name:
      for filename in os.listdir(ds_type):
        if filename.endswith(".TextGrid"):
          tg_files.append(ds_type + "/" + filename)
  print("Total Number of TextGrid Files in Dataset:", len(tg_files))

  # Storing TextGrid Filepaths, subdirectory name, and TextGrid Filenames into separate columns
  df_files = pd.DataFrame(tg_files, columns = ['filename'])
  print(df_files.shape)
  df_files[['dir_name', 'pure_name']] = df_files['filename'].str.split('/', expand = True)
  return df_files

In [26]:
# Example with Single Directory where the Audio Files & Associated TextGrids are saved
df_files = create_df_files("Audio_SR_22050_Sample")
df_files.head()

Total Number of TextGrid Files in Dataset: 30
(30, 1)


,filename,dir_name,pure_name
0,Audio_SR_22050_Sample/5444038033_h_00.TextGrid,Audio_SR_22050_Sample,5444038033_h_00.TextGrid
1,Audio_SR_22050_Sample/0801081005_h_00.TextGrid,Audio_SR_22050_Sample,0801081005_h_00.TextGrid
2,Audio_SR_22050_Sample/5814069018_h_00.TextGrid,Audio_SR_22050_Sample,5814069018_h_00.TextGrid
3,Audio_SR_22050_Sample/5564042025_h_00.TextGrid,Audio_SR_22050_Sample,5564042025_h_00.TextGrid
4,Audio_SR_22050_Sample/5374045014_h_00.TextGrid,Audio_SR_22050_Sample,5374045014_h_00.TextGrid


In [32]:
# Example with Multiple Directories where the Audio Files & Associated TextGrids are saved
df_files = create_df_files(["Train_Sample", "Val_Sample", "Test_Sample"])
df_files.head()

Total Number of TextGrid Files in Dataset: 30
(30, 1)


,filename,dir_name,pure_name
0,Train_Sample/Copy of 0912075002_h_01.TextGrid,Train_Sample,Copy of 0912075002_h_01.TextGrid
1,Train_Sample/Copy of 0801081005_h_00.TextGrid,Train_Sample,Copy of 0801081005_h_00.TextGrid
2,Train_Sample/Copy of 0712054038_h_00.TextGrid,Train_Sample,Copy of 0712054038_h_00.TextGrid
3,Train_Sample/Copy of 0642071018_h_00.TextGrid,Train_Sample,Copy of 0642071018_h_00.TextGrid
4,Train_Sample/Copy of 0642071006_h_00.TextGrid,Train_Sample,Copy of 0642071006_h_00.TextGrid


In [34]:
# Check if the textgrid files themselves are unique
len(df_files['pure_name'].unique()) == df_files.shape[0]

True

In [21]:
import librosa
import soundfile as sf
from scipy import signal
import shutil

def dwnsample_audio(df_files, target_dir, current_sr, target_sr):
  """ Verifies each input audio matches the expected sample rate, then downsamples the audio to the target sample rate. Saves the resampled audio & associated textgrid to the target directory.
      df_files contains a dataframe with columns for TextGrid relative path, subdirectory/subdirectories containing files, TextGrid name for each TextGrid
      Function handles both single and multiple directories storing audio files and saves to a target directory or subdirectories of the target directory
  """
  # If saving to multiple subdirectories within target directory and those subdirectories havent been created yet
  if len(np.unique(df_files['dir_name'])) > 1:
    for sub_dir in np.unique(df_files['dir_name']):
      sub_dir_path = os.path.join(target_dir, sub_dir)
      if not os.path.exists(sub_dir_path):
        print(sub_dir_path)
        os.makedirs(sub_dir_path)

  # Load the audio signal
  files_recheck = []
  for index_pos in range(0, df_files.shape[0]):

    # Loading in each audio file
    textgrid_abspath, sub_dir, tg_pure_name = df_files.iloc[index_pos]
    wav_path = textgrid_abspath.replace("TextGrid", "wav")
    y, sr  = sf.read(wav_path)

    # Downsampling the audio signal to the target sampling rate & verifying current file's sample rate matches expected value for the current sample rate
    if sr == current_sr:
      target_sr = target_sr

      # Resampling the audio signal to the target sampling rate
      y_resampled = signal.resample(y, int(len(y) * target_sr / sr))

      # Saving the resampled audio file & associated textgrid
      if len(np.unique(df_files['dir_name'])) == 1: # Files are all saved in one directory
        wavsave_location = target_dir + "/" + tg_pure_name.replace("TextGrid", "wav")
        tgsave_location = target_dir + "/" + tg_pure_name
      else: # Files are saved in separate directories that have just been created
        wavsave_location = target_dir + "/" + sub_dir + "/" + tg_pure_name.replace("TextGrid", "wav")
        tgsave_location = target_dir + "/" + sub_dir + "/" + tg_pure_name

      # Saving the resampled audio file & associated textgrid
      sf.write(wavsave_location, y_resampled, target_sr)
      shutil.copy(textgrid_abspath, tgsave_location)

    else:
      print(wav_path)
      files_recheck.append(wav_path)

In [22]:
# Calling the above function
target_dir = "Audio_SR_16000"
dwnsample_audio(df_files, target_dir, 22050, 16000)

Audio_SR_16000/Test_Sample
Audio_SR_16000/Train_Sample
Audio_SR_16000/Val_Sample


Verification that All Files Were Correctly Downsampled

In [35]:
def verify_dwnsample(df_files, target_dir):
  if len(np.unique(df_files['dir_name'])) == 1: # If files are saved to target directory
    ds_count = sum([("TextGrid") in filename for filename in os.listdir(target_dir + "/")])
    print(ds_count == df_files.shape[0])
  else:
    counter = 0
    for ds_type in df_files['dir_name'].unique(): # If files are saved to subdirectories of the target directory
      ds_count = sum([("TextGrid") in filename for filename in os.listdir(target_dir + '/' + ds_type)])
      print(ds_type + ":", ds_count)
      counter += ds_count
    print(counter == df_files.shape[0])
verify_dwnsample(df_files, target_dir)

Train_Sample: 10
Val_Sample: 10
Test_Sample: 10
True


### Place for concatenating DataFrame of New TextGrid Filepaths With Other Information: Gender, Target Class, required for creating chunk mapping

In [30]:
df_newdir_paths = target_dir + '/' + df_files['pure_name']
print(df_newdir_paths.shape)
df_newdir_paths.head()

(30,)


0    Audio_SR_16000/5444038033_h_00.TextGrid
1    Audio_SR_16000/0801081005_h_00.TextGrid
2    Audio_SR_16000/5814069018_h_00.TextGrid
3    Audio_SR_16000/5564042025_h_00.TextGrid
4    Audio_SR_16000/5374045014_h_00.TextGrid
Name: pure_name, dtype: object